## Importing Packages

In [3]:
from typing import List, Dict, Optional, Set
from dataclasses import dataclass, asdict, field
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
from datetime import datetime
import random
import time

### Scraping General Market Listings with Features

This code scrapes data from the general market listing page. There are $50$ web pages with $25$ properties listed per page.

There are $34$ facilities available and are represented by columns. These facilities are contained in a list of strings in the source code. If a facility is detected in the list, the code returns $1$. Otherwise, it returns $0$.

In [5]:
@dataclass
class PropertyListing:
    """Data class to store property listing information"""
    title: str
    price: str = None
    price_per_sqm: str = None
    location: str = None
    description: str = None
    bedrooms: str = None
    bathrooms: str = None
    floor_area: str = None
    property_type: str = None
    listing_url: str = None
    agent_name: str = None
    agency_name: str = None
    facilities_dict: Dict[str, int] = field(default_factory=dict)

class DotPropertyScraper:
    """A class to scrape property listings from Dot Property Philippines"""
    
    BASE_URL = "https://www.dotproperty.com.ph/condos-for-sale/metro-manila/quezon-city"
    
    def __init__(self):
        self.session = requests.Session()
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
            'Accept-Language': 'en-US,en;q=0.5'
        }
        self.session.headers.update(self.headers)
        self.all_facilities = set()  # Track all unique facilities

    def _extract_text(self, element, selector, strip=True):
        """Helper method to extract text from HTML elements"""
        try:
            found = element.select_one(selector)
            if found:
                return found.get_text(strip=strip)
            return None
        except Exception:
            return None

    def _extract_facilities(self, article) -> Set[str]:
        """Extract facilities/amenities from the listing"""
        facilities = set()
        try:
            facility_items = article.select('ul.facilities li')
            for item in facility_items:
                facility = item.get_text(strip=True)
                if facility and not facility.startswith('+'):  # Skip the "+X more" text
                    facilities.add(facility)
                    self.all_facilities.add(facility)  # Add to global set
        except Exception:
            pass
        return facilities

    def _create_facilities_dict(self, facilities: Set[str]) -> Dict[str, int]:
        """Create a dictionary of all facilities with 1/0 values"""
        return {facility: 1 if facility in facilities else 0 
                for facility in self.all_facilities}

    def scrape_listings(self, url: str) -> list[PropertyListing]:
        """Scrape property listings from a page"""
        try:
            print(f"Scraping URL: {url}")
            response = self.session.get(url)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')
            
            listings = []
            articles = soup.select('article.listing-snippet')
            
            for article in articles:
                try:
                    # Extract basic information
                    title = self._extract_text(article, '.text-2xl.font-semibold')
                    price = self._extract_text(article, '.text-secondary-base.whitespace-nowrap.font-bold.text-3xl')
                    price_per_sqm = self._extract_text(article, '.text-neutral-2.whitespace-nowrap.font-bold.text-xl')
                    location = self._extract_text(article, '.text-neutral-2.my-[5px]')
                    description = self._extract_text(article, '.line-clamp-4')
                    
                    # Extract property details
                    details = article.select('ul.list-none.p-0.flex.gap-x-5 li')
                    bedrooms = bathrooms = floor_area = property_type = None
                    
                    for detail in details:
                        text = detail.get_text(strip=True)
                        img = detail.select_one('img')
                        if img:
                            if 'bed' in img.get('src', ''):
                                bedrooms = text
                            elif 'bathtub' in img.get('src', ''):
                                bathrooms = text
                            elif 'resize' in img.get('src', ''):
                                floor_area = text
                            elif 'home' in img.get('src', ''):
                                property_type = text

                    # Extract agent/agency information
                    agent_name = self._extract_text(article, '.font-bold.truncate.w-full.text-base.text-neutral-2')
                    agency_div = article.select_one('.col-span-1.grid.grid-cols-3.gap-3')
                    agency_name = agency_div.get('title') if agency_div else None

                    # Get listing URL
                    listing_url = article.select_one('a[href*="/ads/"]')
                    listing_url = listing_url.get('href') if listing_url else None
                    
                    # Extract facilities
                    listing_facilities = self._extract_facilities(article)
                    
                    # Create listing object
                    listing = PropertyListing(
                        title=title,
                        price=price,
                        price_per_sqm=price_per_sqm,
                        location=location,
                        description=description,
                        bedrooms=bedrooms,
                        bathrooms=bathrooms,
                        floor_area=floor_area,
                        property_type=property_type,
                        listing_url=listing_url,
                        agent_name=agent_name,
                        agency_name=agency_name,
                        facilities_dict={}  # Will be populated after all scraping is done
                    )
                    
                    listings.append((listing, listing_facilities))  # Store facilities separately for now
                    
                except Exception as e:
                    print(f"Error parsing listing: {e}")
                    continue
                
            return listings
            
        except requests.RequestException as e:
            print(f"Error fetching data: {e}")
            return []
        except Exception as e:
            print(f"Unexpected error: {e}")
            return []

    def save_to_csv(self, listings_with_facilities: list[tuple[PropertyListing, Set[str]]], filename: str = None):
        """Save the listings to a CSV file with separate facility columns"""
        if not listings_with_facilities:
            print("No listings to save.")
            return
        
        # Now that we have all facilities, create the facilities dict for each listing
        listings_data = []
        for listing, facilities in listings_with_facilities:
            listing_dict = asdict(listing)
            del listing_dict['facilities_dict']  # Remove the placeholder dict
            
            # Add facility columns
            for facility in self.all_facilities:
                listing_dict[f"facility_{facility.lower().replace(' ', '_')}"] = 1 if facility in facilities else 0
            
            listings_data.append(listing_dict)
        
        if filename is None:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"dotproperty_listings_{timestamp}.csv"
        
        df = pd.DataFrame(listings_data)
        df.to_csv(filename, index=False, encoding='utf-8-sig')
        print(f"Saved {len(listings_data)} listings to {filename}")
        print(f"Total unique facilities found: {len(self.all_facilities)}")
        print("Facility columns created:", sorted(self.all_facilities))

def main():
    """Example usage of the Dot Property Scraper"""
    scraper = DotPropertyScraper()
    base_url = "https://www.dotproperty.com.ph/condos-for-sale/metro-manila/quezon-city?exact_bed=false"
    
    all_listings = []
    
    # Scrape first page
    listings = scraper.scrape_listings(base_url)
    if listings:
        all_listings.extend(listings)
        print(f"Found {len(listings)} listings on page 1")
    
    # Scrape subsequent pages
    page = 2
    while listings and page <= 100:  # Limit to 100 pages
        page_url = f"{base_url}&page={page}"
        listings = scraper.scrape_listings(page_url)
        
        if listings:
            all_listings.extend(listings)
            print(f"Found {len(listings)} listings on page {page}")
        else:
            print(f"No listings found on page {page}")
            break
            
        page += 1
        time.sleep(random.uniform(2, 4))  # Random delay between requests
    
    if all_listings:
        scraper.save_to_csv(all_listings)
    
    print(f"\nTotal listings scraped: {len(all_listings)}")

if __name__ == "__main__":
    main()

Scraping URL: https://www.dotproperty.com.ph/condos-for-sale/metro-manila/quezon-city?exact_bed=false
Found 25 listings on page 1
Scraping URL: https://www.dotproperty.com.ph/condos-for-sale/metro-manila/quezon-city?exact_bed=false&page=2
Found 25 listings on page 2
Scraping URL: https://www.dotproperty.com.ph/condos-for-sale/metro-manila/quezon-city?exact_bed=false&page=3
Found 25 listings on page 3
Scraping URL: https://www.dotproperty.com.ph/condos-for-sale/metro-manila/quezon-city?exact_bed=false&page=4
Found 25 listings on page 4
Scraping URL: https://www.dotproperty.com.ph/condos-for-sale/metro-manila/quezon-city?exact_bed=false&page=5
Found 25 listings on page 5
Scraping URL: https://www.dotproperty.com.ph/condos-for-sale/metro-manila/quezon-city?exact_bed=false&page=6
Found 25 listings on page 6
Scraping URL: https://www.dotproperty.com.ph/condos-for-sale/metro-manila/quezon-city?exact_bed=false&page=7
Found 25 listings on page 7
Scraping URL: https://www.dotproperty.com.ph/con

### Scraping General Marketing Listings with Geolocation

The source code from the general market listings does not contain the location of the property. This information is contained in the source code of the market listing itself. To resolve this, the code scrapes informatiton in the general listing page then uses the `url` feature to go to the specific marketing pages to retrieve other features such as geolocation. The new dataset also added $2$ more facilities.

In [9]:
from dataclasses import dataclass, asdict, field
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
from datetime import datetime
import random
import time
from typing import Set, Dict
import re

@dataclass
class PropertyListing:
    """Data class to store property listing information"""
    title: str
    price: str = None
    price_per_sqm: str = None
    location: str = None
    description: str = None
    bedrooms: str = None
    bathrooms: str = None
    floor_area: str = None
    floor_number: str = None
    property_type: str = None
    listing_url: str = None
    agent_name: str = None
    agency_name: str = None
    latitude: str = None
    longitude: str = None
    facilities_dict: Dict[str, int] = field(default_factory=dict)

class DotPropertyScraper:
    """A class to scrape property listings from Dot Property Philippines"""
    
    BASE_URL = "https://www.dotproperty.com.ph"
    
    def __init__(self):
        self.session = requests.Session()
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
            'Accept-Language': 'en-US,en;q=0.5'
        }
        self.session.headers.update(self.headers)
        self.all_facilities = set()

    def _extract_text(self, element, selector, strip=True):
        """Helper method to extract text from HTML elements"""
        try:
            found = element.select_one(selector)
            if found:
                return found.get_text(strip=strip)
            return None
        except Exception:
            return None

    def _extract_coordinates(self, html_content):
        """Extract GPS coordinates from the page source"""
        lat = lon = None
        try:
            lat_match = re.search(r'var gps_lat = "([^"]+)"', html_content)
            lon_match = re.search(r'var gps_lon = "([^"]+)"', html_content)
            if lat_match:
                lat = lat_match.group(1)
            if lon_match:
                lon = lon_match.group(1)
        except Exception as e:
            print(f"Error extracting coordinates: {e}")
        return lat, lon

    def _extract_listing_details(self, url):
        """Extract detailed information from individual listing page"""
        try:
            response = self.session.get(url)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Extract facilities from the detailed page
            facilities = set()
            facility_elements = soup.select('.facilities li, .amenities li')
            for elem in facility_elements:
                facility = elem.get_text(strip=True)
                if facility and not facility.startswith('+'):
                    facilities.add(facility)
                    self.all_facilities.add(facility)

            # Extract floor number
            floor_number = None
            key_features = soup.select('.key-featured li')
            for feature in key_features:
                text = feature.get_text(strip=True)
                if 'Floor:' in text:
                    floor_number = text.split(':')[-1].strip()
                    break

            # Extract coordinates
            lat, lon = self._extract_coordinates(response.text)
            
            return {
                'facilities': facilities,
                'floor_number': floor_number,
                'latitude': lat,
                'longitude': lon
            }
            
        except Exception as e:
            print(f"Error fetching listing details: {e}")
            return {'facilities': set(), 'floor_number': None, 'latitude': None, 'longitude': None}

    def scrape_listings(self, url: str) -> list[PropertyListing]:
        """Scrape property listings from a page"""
        try:
            print(f"Scraping URL: {url}")
            response = self.session.get(url)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')
            
            listings = []
            articles = soup.select('article.listing-snippet')
            
            for article in articles:
                try:
                    # Extract basic information
                    title = self._extract_text(article, '.text-2xl.font-semibold')
                    price = self._extract_text(article, '.text-secondary-base.whitespace-nowrap.font-bold.text-3xl')
                    price_per_sqm = self._extract_text(article, '.text-neutral-2.whitespace-nowrap.font-bold.text-xl')
                    location = self._extract_text(article, '.text-neutral-2.my-[5px]')
                    description = self._extract_text(article, '.line-clamp-4')
                    
                    # Extract property details
                    details = article.select('ul.list-none.p-0.flex.gap-x-5 li')
                    bedrooms = bathrooms = floor_area = property_type = None
                    
                    for detail in details:
                        text = detail.get_text(strip=True)
                        img = detail.select_one('img')
                        if img:
                            if 'bed' in img.get('src', ''):
                                bedrooms = text
                            elif 'bathtub' in img.get('src', ''):
                                bathrooms = text
                            elif 'resize' in img.get('src', ''):
                                floor_area = text
                            elif 'home' in img.get('src', ''):
                                property_type = text

                    # Extract agent/agency information
                    agent_name = self._extract_text(article, '.font-bold.truncate.w-full.text-base.text-neutral-2')
                    agency_div = article.select_one('.col-span-1.grid.grid-cols-3.gap-3')
                    agency_name = agency_div.get('title') if agency_div else None

                    # Get listing URL and fetch detailed information
                    listing_url_elem = article.select_one('a[href*="/ads/"]')
                    if listing_url_elem:
                        listing_url = listing_url_elem.get('href')
                        if not listing_url.startswith('http'):
                            listing_url = f"{self.BASE_URL}{listing_url}"
                            
                        print(f"Fetching details from: {listing_url}")
                        details = self._extract_listing_details(listing_url)
                        time.sleep(random.uniform(1, 2))  # Random delay between requests
                    else:
                        details = {'facilities': set(), 'floor_number': None, 'latitude': None, 'longitude': None}
                        listing_url = None
                    
                    # Create listing object
                    listing = PropertyListing(
                        title=title,
                        price=price,
                        price_per_sqm=price_per_sqm,
                        location=location,
                        description=description,
                        bedrooms=bedrooms,
                        bathrooms=bathrooms,
                        floor_area=floor_area,
                        floor_number=details['floor_number'],
                        property_type=property_type,
                        listing_url=listing_url,
                        agent_name=agent_name,
                        agency_name=agency_name,
                        latitude=details['latitude'],
                        longitude=details['longitude'],
                        facilities_dict={}  # Will be populated after all scraping is done
                    )
                    
                    listings.append((listing, details['facilities']))
                    
                except Exception as e:
                    print(f"Error parsing listing: {e}")
                    continue
                
            return listings
            
        except requests.RequestException as e:
            print(f"Error fetching data: {e}")
            return []
        except Exception as e:
            print(f"Unexpected error: {e}")
            return []

    def save_to_csv(self, listings_with_facilities: list[tuple[PropertyListing, Set[str]]], filename: str = None):
        """Save the listings to a CSV file with separate facility columns"""
        if not listings_with_facilities:
            print("No listings to save.")
            return
        
        # Now that we have all facilities, create the facilities dict for each listing
        listings_data = []
        for listing, facilities in listings_with_facilities:
            listing_dict = asdict(listing)
            del listing_dict['facilities_dict']  # Remove the placeholder dict
            
            # Add facility columns
            for facility in self.all_facilities:
                listing_dict[f"facility_{facility.lower().replace(' ', '_')}"] = 1 if facility in facilities else 0
            
            listings_data.append(listing_dict)
        
        if filename is None:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"dotproperty_listings_{timestamp}.csv"
        
        df = pd.DataFrame(listings_data)
        df.to_csv(filename, index=False, encoding='utf-8-sig')
        print(f"Saved {len(listings_data)} listings to {filename}")
        print(f"Total unique facilities found: {len(self.all_facilities)}")
        print("Facility columns created:", sorted(self.all_facilities))

def main():
    """Example usage of the Dot Property Scraper"""
    scraper = DotPropertyScraper()
    base_url = "https://www.dotproperty.com.ph/condos-for-sale/metro-manila/quezon-city?exact_bed=false"
    
    all_listings = []
    
    # Scrape first page
    listings = scraper.scrape_listings(base_url)
    if listings:
        all_listings.extend(listings)
        print(f"Found {len(listings)} listings on page 1")
    
    # Scrape subsequent pages
    page = 2
    while listings and page <= 100:  # Limit to 100 pages
        page_url = f"{base_url}&page={page}"
        listings = scraper.scrape_listings(page_url)
        
        if listings:
            all_listings.extend(listings)
            print(f"Found {len(listings)} listings on page {page}")
        else:
            print(f"No listings found on page {page}")
            break
            
        page += 1
        time.sleep(random.uniform(2, 4))  # Random delay between requests
    
    if all_listings:
        scraper.save_to_csv(all_listings)
    
    print(f"\nTotal listings scraped: {len(all_listings)}")

if __name__ == "__main__":
    main()

Scraping URL: https://www.dotproperty.com.ph/condos-for-sale/metro-manila/quezon-city?exact_bed=false
Fetching details from: https://www.dotproperty.com.ph/ads/1-bedroom-condo-for-sale-in-the-crestmont-south-triangle-metro-manila-near-mrt-3-quezon-avenue_1dd35619f653-2470-1332-e97c-7b607089
Fetching details from: https://www.dotproperty.com.ph/ads/1-bedroom-condo-for-sale-in-the-crestmont-south-triangle-metro-manila-near-mrt-3-quezon-avenue_cc46dd081316-0521-e9a5-fa04-bdf6e1a3
Fetching details from: https://www.dotproperty.com.ph/ads/2-bedroom-condo-for-sale-in-marilag-metro-manila-near-lrt-2-anonas_0afb1dbd674f-1b6f-c5a8-4b43-940a6785
Fetching details from: https://www.dotproperty.com.ph/ads/2-bedroom-condo-for-sale-in-the-crestmont-south-triangle-metro-manila-near-mrt-3-quezon-avenue_a2db06aa3b1d-d941-c992-e97b-6f5a8089
Fetching details from: https://www.dotproperty.com.ph/ads/2-bedroom-condo-for-sale-in-viera-residences-obrero-metro-manila_8f58a25e79fe-9ace-1662-073e-b0fb8089
Fetchi

Fetching details from: https://www.dotproperty.com.ph/ads/1-bedroom-condo-for-sale-in-primehomes-capitol-hills-matandang-balara-metro-manila_3be1a518d65a-c6d1-1aa2-6c5e-02007089
Found 25 listings on page 2
Scraping URL: https://www.dotproperty.com.ph/condos-for-sale/metro-manila/quezon-city?exact_bed=false&page=3
Fetching details from: https://www.dotproperty.com.ph/ads/2-bedroom-condo-for-sale-in-matandang-balara-metro-manila_6d7535f24b68-9b90-a9f2-7b3c-bddc7089
Fetching details from: https://www.dotproperty.com.ph/ads/1-bedroom-condo-for-sale-in-primehomes-capitol-hills-matandang-balara-metro-manila_57d4ce43f5a4-93ef-1442-50d2-94ce7089
Fetching details from: https://www.dotproperty.com.ph/ads/1-bedroom-condo-for-sale-in-primehomes-capitol-hills-matandang-balara-metro-manila_986904bdf625-93ef-1442-4d76-d5ce7089
Fetching details from: https://www.dotproperty.com.ph/ads/1-bedroom-condo-for-sale-in-primehomes-capitol-hills-matandang-balara-metro-manila_3cfc7f68c218-7b0e-f552-66cd-de11708

Fetching details from: https://www.dotproperty.com.ph/ads/1-bedroom-condo-for-sale-in-primehomes-capitol-hills-matandang-balara-metro-manila_5764eb23c4a9-b6d1-1aa2-b61c-be007089
Fetching details from: https://www.dotproperty.com.ph/ads/6-bedroom-condo-for-sale-in-kaunlaran-metro-manila-near-lrt-2-betty-go-belmonte_df492f9c4e4a-0510-6422-cde8-854b7089
Fetching details from: https://www.dotproperty.com.ph/ads/1-bedroom-condo-for-sale-in-primehomes-capitol-hills-matandang-balara-metro-manila_ecd8c5d6a82e-93ef-1442-27f2-03ce7089
Fetching details from: https://www.dotproperty.com.ph/ads/1-bedroom-condo-for-sale-in-primehomes-capitol-hills-matandang-balara-metro-manila_052c1ef3e8af-93ef-1442-e00c-83ce7089
Fetching details from: https://www.dotproperty.com.ph/ads/1-bedroom-condo-for-sale-in-primehomes-capitol-hills-matandang-balara-metro-manila_eaf844cc752f-93ef-1442-1044-93ce7089
Found 25 listings on page 4
Scraping URL: https://www.dotproperty.com.ph/condos-for-sale/metro-manila/quezon-city

Fetching details from: https://www.dotproperty.com.ph/ads/condo-for-sale-in-san-roque-metro-manila-near-lrt-2-anonas_862bee497b8c-0180-6092-bca4-74e37089
Fetching details from: https://www.dotproperty.com.ph/ads/1-bedroom-condo-for-sale-in-socorro-metro-manila-near-lrt-2-araneta-center-cubao_9100faad6f1d-4fbf-f652-8902-ee968089
Found 25 listings on page 6
Scraping URL: https://www.dotproperty.com.ph/condos-for-sale/metro-manila/quezon-city?exact_bed=false&page=7
Fetching details from: https://www.dotproperty.com.ph/ads/1-bedroom-condo-for-sale-in-shang-summit-south-triangle-metro-manila-near-mrt-3-kamuning_94b9a0f32861-4e61-e992-12f9-c2976089
Fetching details from: https://www.dotproperty.com.ph/ads/1-bedroom-condo-for-sale-in-haraya-residences-blue-ridge-b-metro-manila_0c24ea81f475-3d7f-bbb2-c8dc-eab86089
Fetching details from: https://www.dotproperty.com.ph/ads/1-bedroom-condo-for-sale-in-haraya-residences-blue-ridge-b-metro-manila_6acf90853916-3d7f-bbb2-d15f-5ab86089
Fetching detail

Fetching details from: https://www.dotproperty.com.ph/ads/condo-for-sale-in-primehomes-capitol-hills-matandang-balara-metro-manila_be58c548a011-f600-2ee2-1a4f-20877089
Fetching details from: https://www.dotproperty.com.ph/ads/condo-for-sale-in-primehomes-capitol-hills-matandang-balara-metro-manila_af3c62b3ed31-f600-2ee2-0dd8-30877089
Fetching details from: https://www.dotproperty.com.ph/ads/condo-for-sale-in-primehomes-capitol-hills-matandang-balara-metro-manila_f852f0d88af2-f600-2ee2-5ca4-50877089
Fetching details from: https://www.dotproperty.com.ph/ads/condo-for-sale-in-primehomes-capitol-hills-matandang-balara-metro-manila_3edbe1738352-f600-2ee2-7c1d-80877089
Fetching details from: https://www.dotproperty.com.ph/ads/condo-for-sale-in-primehomes-capitol-hills-matandang-balara-metro-manila_a8c9b374bfd3-f600-2ee2-1d7c-a1877089
Fetching details from: https://www.dotproperty.com.ph/ads/condo-for-sale-in-primehomes-capitol-hills-matandang-balara-metro-manila_1bd0ca4f8163-f600-2ee2-fa49-b

...

Fetching details from: https://www.dotproperty.com.ph/ads/3-bedroom-condo-for-sale-in-ugong-norte-metro-manila_c7178d596057-bc6f-c115-d26c-1912d497
Found 25 listings on page 40
Scraping URL: https://www.dotproperty.com.ph/condos-for-sale/metro-manila/quezon-city?exact_bed=false&page=41
Fetching details from: https://www.dotproperty.com.ph/ads/condo-for-sale-in-bagong-pag-asa-metro-manila-near-mrt-3-north-avenue_ccb31ce1e68c-8aa1-8e95-8599-754c5fc7
Fetching details from: https://www.dotproperty.com.ph/ads/condo-for-sale-in-mira-san-roque-metro-manila-near-lrt-2-anonas_e392bdacc326-75c0-a6c5-6e88-17b216a0
Fetching details from: https://www.dotproperty.com.ph/ads/condo-for-sale-in-mira-san-roque-metro-manila-near-lrt-2-anonas_35bf9fbb161a-87ff-efd5-fa24-8cc2dbc3
Fetching details from: https://www.dotproperty.com.ph/ads/condo-for-sale-in-mira-san-roque-metro-manila-near-lrt-2-anonas_f7e335d61bd5-ce71-0be5-8b7a-cd504054
Fetching details from: https://www.dotproperty.com.ph/ads/condo-for-sal

Fetching details from: https://www.dotproperty.com.ph/ads/3-bedroom-condo-for-sale-in-damayang-lagi-metro-manila_3731e5b7f640-070f-14a8-95f2-98f6db01
Fetching details from: https://www.dotproperty.com.ph/ads/2-bedroom-condo-for-sale-in-the-orabella-pasong-tamo-metro-manila_30eac51dd289-8d9e-cff2-80ba-439e9089
Fetching details from: https://www.dotproperty.com.ph/ads/1-bedroom-condo-for-sale-in-sentrove-at-cloverleaf-balingasa-metro-manila-near-lrt-1-balintawak_2c2c5426d42f-9111-0225-16a4-8713ca36
Fetching details from: https://www.dotproperty.com.ph/ads/1-bedroom-condo-for-sale-in-bagumbayan-metro-manila_3e7c13f74a76-53d0-f615-c5f2-220b2751
Fetching details from: https://www.dotproperty.com.ph/ads/1-bedroom-condo-for-sale-in-bagumbayan-metro-manila_249d8833c86c-ad80-ead5-219e-e141971a
Fetching details from: https://www.dotproperty.com.ph/ads/1-bedroom-condo-for-sale-in-loyola-heights-metro-manila-near-lrt-2-katipunan_fca0647a76ef-fa2f-04a8-bd9c-95fda767
Fetching details from: https://w

Fetching details from: https://www.dotproperty.com.ph/ads/1-bedroom-condo-for-sale-in-bagumbayan-metro-manila_7872f8894f47-760e-4075-f4b9-d52415fa
Fetching details from: https://www.dotproperty.com.ph/ads/1-bedroom-condo-for-sale-in-valencia-metro-manila-near-lrt-2-gilmore_deccab7d4ad8-8b9e-e6d2-9d43-36b77089
Fetching details from: https://www.dotproperty.com.ph/ads/condo-for-sale-in-mira-san-roque-metro-manila-near-lrt-2-anonas_8366324f1af5-8d00-6442-ab08-473f8089
Fetching details from: https://www.dotproperty.com.ph/ads/3-bedroom-condo-for-sale-in-san-isidro-metro-manila_d6e2224fefcf-1241-cb88-8f3e-919632be
Fetching details from: https://www.dotproperty.com.ph/ads/condo-for-sale-in-the-oriana-marilag-metro-manila-near-lrt-2-anonas_6d0f02731401-406f-4112-3ae9-52747089
Fetching details from: https://www.dotproperty.com.ph/ads/1-bedroom-condo-for-sale-in-the-magnolia-residences-tower-a-b-and-c-kaunlaran-metro-manila-near-lrt-2-gilmore_68b0d292c581-1c4e-f332-29f0-094c8089
Fetching detail

Fetching details from: https://www.dotproperty.com.ph/ads/condo-for-sale-in-the-oriana-marilag-metro-manila-near-lrt-2-anonas_44dde337eea0-406f-4112-1776-b2747089
Fetching details from: https://www.dotproperty.com.ph/ads/condo-for-sale-in-the-oriana-marilag-metro-manila-near-lrt-2-anonas_34d15c105ae7-406f-4112-bbd4-5d847089
Fetching details from: https://www.dotproperty.com.ph/ads/1-bedroom-condo-for-sale-in-infina-towers-marilag-metro-manila-near-lrt-2-anonas_90a113f9f506-4eae-3bb2-e8fd-becf7089
Fetching details from: https://www.dotproperty.com.ph/ads/2-bedroom-condo-for-sale-in-infina-towers-marilag-metro-manila-near-lrt-2-anonas_61f67943c22f-0530-1dd2-7efb-f0718089
Fetching details from: https://www.dotproperty.com.ph/ads/condo-for-sale-in-south-triangle-metro-manila-near-mrt-3-quezon-avenue_bfc5c545e7ea-43e0-f055-6062-fa6a635b
Fetching details from: https://www.dotproperty.com.ph/ads/3-bedroom-condo-for-sale-in-bagumbayan-metro-manila_c8ca2622f6b8-7151-0552-c956-0ff48089
Fetching 

Fetching details from: https://www.dotproperty.com.ph/ads/2-bedroom-condo-for-sale-in-quezon-city-metro-manila-near-lrt-1-roosevelt_dd798644e856-b49f-b442-e8f1-28528089
Fetching details from: https://www.dotproperty.com.ph/ads/condo-for-sale-in-loyola-heights-metro-manila-near-lrt-2-katipunan_8c4d2dd647cc-01b0-2cc2-ff16-65107089
Fetching details from: https://www.dotproperty.com.ph/ads/3-bedroom-condo-for-sale-in-ugong-norte-metro-manila_80241a042091-477f-7112-0221-07646089
Fetching details from: https://www.dotproperty.com.ph/ads/2-bedroom-condo-for-sale-in-loyola-heights-metro-manila-near-lrt-2-katipunan_c66f36a5cc1b-db51-9205-b7a2-62ab4bcb
Fetching details from: https://www.dotproperty.com.ph/ads/3-bedroom-condo-for-sale-in-the-crestmont-south-triangle-metro-manila-near-mrt-3-quezon-avenue_4ec811241bcc-fe90-f795-1b9c-0c1103d8
Fetching details from: https://www.dotproperty.com.ph/ads/3-bedroom-condo-for-sale-in-the-crestmont-south-triangle-metro-manila-near-mrt-3-quezon-avenue_c60b4c